In [1]:
import pandas as pd
import sqlite3

In [2]:
db = sqlite3.connect('../../datasets/checking-logs.sqlite')

In [28]:
pd.read_sql('PRAGMA table_info(pageviews);',db)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,uid,TEXT,0,None,0
2,2,datetime,TIMESTAMP,0,None,0


In [43]:
test_results = pd.read_sql("""
    WITH users_all AS (SELECT uid, first_view_ts
    FROM test
    WHERE labname != 'project1'
        AND first_view_ts IS NOT NULL
        AND EXISTS (SELECT 1 FROM test t2 WHERE t2.uid = test.uid AND t2.first_commit_ts < test.first_view_ts) AND EXISTS (SELECT 1 FROM test t3 WHERE t3.uid = test.uid 
        AND t3.first_commit_ts >= test.first_view_ts))
    SELECT CASE WHEN t.first_commit_ts < u.first_view_ts THEN 'before'ELSE 'after' END AS time, AVG((strftime('%s', t.first_commit_ts) - d.deadlines) / 3600) AS avg_diff
    FROM test t
    JOIN deadlines d ON t.labname = d.labs
    JOIN users_all u ON t.uid = u.uid
    WHERE t.labname != 'project1'
    GROUP BY time
""", db)
test_results

,time,avg_diff
0,after,-96.354839
1,before,-61.068493


In [44]:
control_results = pd.read_sql(""" 
    WITH users_all AS (
    SELECT uid, first_view_ts
    FROM control
    WHERE labname != 'project1'
        AND first_view_ts IS NOT NULL
        AND EXISTS ( SELECT 1 FROM control c2 WHERE c2.uid = control.uid AND c2.first_commit_ts < control.first_view_ts)
        AND EXISTS (SELECT 1 FROM control c3 WHERE c3.uid = control.uid AND c3.first_commit_ts >= control.first_view_ts))
    SELECT CASE WHEN c.first_commit_ts < u.first_view_ts THEN 'before' ELSE 'after' END AS time, AVG((strftime('%s', c.first_commit_ts) - d.deadlines) / 3600) AS avg_diff
    FROM control c
    JOIN deadlines d ON c.labname = d.labs
    JOIN users_all u ON c.uid = u.uid
    WHERE c.labname != 'project1'
    GROUP BY time
    """, db)
control_results

,time,avg_diff
0,after,-127.205479
1,before,-110.641026


In [47]:
db.close()